In [1]:
// Breast Cancer Wisconsin (Diagnostic) Database
//
// Notes
// -----
// Data Set Characteristics:
//     :Number of Instances: 569
//
//     :Number of Attributes: 30 numeric, predictive attributes and the class
//
//     :Attribute Information:
//         - radius (mean of distances from center to points on the perimeter)
//         - texture (standard deviation of gray-scale values)
//         - perimeter
//         - area
//         - smoothness (local variation in radius lengths)
//         - compactness (perimeter^2 / area - 1.0)
//         - concavity (severity of concave portions of the contour)
//         - concave points (number of concave portions of the contour)
//         - symmetry
//         - fractal dimension ("coastline approximation" - 1)
//
//         The mean, standard error, and "worst" or largest (mean of the three
//         largest values) of these features were computed for each image,
//         resulting in 30 features.  For instance, field 3 is Mean Radius, field
//         13 is Radius SE, field 23 is Worst Radius.
//
//         - class:
//                 - WDBC-Malignant
//                 - WDBC-Benign
//
//     :Summary Statistics:
//
//     ===================================== ======= ========
//                                            Min     Max
//     ===================================== ======= ========
//     radius (mean):                         6.981   28.11
//     texture (mean):                        9.71    39.28
//     perimeter (mean):                      43.79   188.5
//     area (mean):                           143.5   2501.0
//     smoothness (mean):                     0.053   0.163
//     compactness (mean):                    0.019   0.345
//     concavity (mean):                      0.0     0.427
//     concave points (mean):                 0.0     0.201
//     symmetry (mean):                       0.106   0.304
//     fractal dimension (mean):              0.05    0.097
//     radius (standard error):               0.112   2.873
//     texture (standard error):              0.36    4.885
//     perimeter (standard error):            0.757   21.98
//     area (standard error):                 6.802   542.2
//     smoothness (standard error):           0.002   0.031
//     compactness (standard error):          0.002   0.135
//     concavity (standard error):            0.0     0.396
//     concave points (standard error):       0.0     0.053
//     symmetry (standard error):             0.008   0.079
//     fractal dimension (standard error):    0.001   0.03
//     radius (worst):                        7.93    36.04
//     texture (worst):                       12.02   49.54
//     perimeter (worst):                     50.41   251.2
//     area (worst):                          185.2   4254.0
//     smoothness (worst):                    0.071   0.223
//     compactness (worst):                   0.027   1.058
//     concavity (worst):                     0.0     1.252
//     concave points (worst):                0.0     0.291
//     symmetry (worst):                      0.156   0.664
//     fractal dimension (worst):             0.055   0.208
//     ===================================== ======= ========
//
//     :Missing Attribute Values: None
//
//     :Class Distribution: 212 - Malignant, 357 - Benign
//
//     :Creator:  Dr. William H. Wolberg, W. Nick Street, Olvi L. Mangasarian
//
//     :Donor: Nick Street
//
//     :Date: November, 1995

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.22:4042
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1601805491317)
SparkSession available as 'spark'


In [2]:
// Import Spark and Create a Spark Session
import org.apache.spark.sql.SparkSession

// Import PCA, VectorAssembler and StandardScaler from ml.feature
import org.apache.spark.ml.feature.{PCA, VectorAssembler, StandardScaler}
// import org.apache.spark.ml.feature.{VectorAssembler,StringIndexer,VectorIndexer,OneHotEncoder}

// Import Vectors from ml.linalg
import org.apache.spark.ml.linalg.Vectors

import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.feature.{PCA, VectorAssembler, StandardScaler}
import org.apache.spark.ml.linalg.Vectors


In [3]:
// Create a Spark session.
val spark = SparkSession.builder().getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5e1638d3


In [4]:
// Use Spark to read in the cancer_data file.
val data = spark.read
    .option("header","true")
    .option("inferSchema","true")
    .format("csv")
    .load("cancer_data")

// Print the schema of the data.
data.printSchema()

root
 |-- mean radius: integer (nullable = true)
 |-- mean texture: double (nullable = true)
 |-- mean perimeter: double (nullable = true)
 |-- mean area: double (nullable = true)
 |-- mean smoothness: double (nullable = true)
 |-- mean compactness: double (nullable = true)
 |-- mean concavity: double (nullable = true)
 |-- mean concave points: double (nullable = true)
 |-- mean symmetry: double (nullable = true)
 |-- mean fractal dimension: double (nullable = true)
 |-- radius error: double (nullable = true)
 |-- texture error: double (nullable = true)
 |-- perimeter error: double (nullable = true)
 |-- area error: double (nullable = true)
 |-- smoothness error: double (nullable = true)
 |-- compactness error: double (nullable = true)
 |-- concavity error: double (nullable = true)
 |-- concave points error: double (nullable = true)
 |-- symmetry error: double (nullable = true)
 |-- fractal dimension error: double (nullable = true)
 |-- worst radius: double (nullable = true)
 |-- worst

data: org.apache.spark.sql.DataFrame = [mean radius: int, mean texture: double ... 28 more fields]


In [5]:
val column_names = data.columns

column_names: Array[String] = Array(mean radius, mean texture, mean perimeter, mean area, mean smoothness, mean compactness, mean concavity, mean concave points, mean symmetry, mean fractal dimension, radius error, texture error, perimeter error, area error, smoothness error, compactness error, concavity error, concave points error, symmetry error, fractal dimension error, worst radius, worst texture, worst perimeter, worst area, worst smoothness, worst compactness, worst concavity, worst concave points, worst symmetry, worst fractal dimension)


In [6]:
// Use VectorAssembler to convert the input columns of the cancer data
// to a single output column of an array called "features"
// Set the input columns from which we are supposed to read the values.
// Call this new object assembler.

val assembler = new VectorAssembler()
  .setInputCols(column_names)
  .setOutputCol("features")

// Use the assembler to transform our DataFrame to a single column: features
val output = assembler.transform(data).select($"features")

assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_6482d95f2f10, handleInvalid=error, numInputCols=30
output: org.apache.spark.sql.DataFrame = [features: vector]


In [7]:
// Often its a good idea to normalize each feature to have unit standard
// deviation and/or zero mean, when using PCA.
// This is essentially a pre-step to PCA, but its not always necessary.

// Look at the ml.feature documentation and figure out how to standardize
// the cancer data set. Refer to the solutions for hints if you get stuck.

// Use StandardScaler on the data
// Create a new StandardScaler() object called scaler
// Set the input to the features column and the ouput to a column called
// scaledFeatures

val scaler = new StandardScaler()
    .setInputCol("features")
    .setOutputCol("scaledFeatures")
    .setWithStd(true)
    .setWithMean(false)

scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_2bacfdb23024


In [8]:
// Compute summary statistics by fitting the StandardScaler.
// Basically create a new object called scalerModel by using scaler.fit()
// on the output of the VectorAssembler
val scalerModel = scaler.fit(output)

// Normalize each feature to have unit standard deviation.
// Use transform() off of this scalerModel object to create your scaledData
val scaledData = scalerModel.transform(output)

scalerModel: org.apache.spark.ml.feature.StandardScalerModel = StandardScalerModel: uid=stdScal_2bacfdb23024, numFeatures=30, withMean=false, withStd=true
scaledData: org.apache.spark.sql.DataFrame = [features: vector, scaledFeatures: vector]


In [9]:
// Now its time to use PCA to reduce the features to some principal components

// Create a new PCA() object that will take in the scaledFeatures
// and output the pcs features, use 4 principal components
// Then fit this to the scaledData
val pca = (new PCA()
    .setInputCol("scaledFeatures")
    .setOutputCol("pcaFeatures")
    .setK(4)
    .fit(scaledData))

pca: org.apache.spark.ml.feature.PCAModel = PCAModel: uid=pca_7669a40b137a, k=4


In [10]:
// Once your pca has been created and fit, transform the scaledData
// Call this new dataframe pcaDF
val pcaDF = pca.transform(scaledData)

// Show the new pcaFeatures
val result = pcaDF.select("pcaFeatures")
result.show()

// Use .head() to confirm that your output column Array of pcaFeatures
// only has 4 principal components
result.head(1)

+--------------------+
|         pcaFeatures|
+--------------------+
|[21.6219973823649...|
|[15.1217370347583...|
|[18.4325856097778...|
|[18.9549565028940...|
|[16.7333072691963...|
|[14.6702143413857...|
|[14.9731218113083...|
|[14.5441067848488...|
|[15.5980062310548...|
|[17.9720563963992...|
|[11.9024481321385...|
|[15.1741616728687...|
|[20.7357773767196...|
|[13.1902413206624...|
|[16.4793766691530...|
|[16.3184525605226...|
|[12.9382787109835...|
|[16.8474451580445...|
|[17.7858229434146...|
|[11.5044884686410...|
+--------------------+
only showing top 20 rows



pcaDF: org.apache.spark.sql.DataFrame = [features: vector, scaledFeatures: vector ... 1 more field]
result: org.apache.spark.sql.DataFrame = [pcaFeatures: vector]
res2: Array[org.apache.spark.sql.Row] = Array([[21.62199738236499,8.516595739466716,-3.731847417579699,-0.41812449701477833]])
